In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import csv

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/transformer')



# Loading the training and test data
with open('/content/drive/MyDrive/transformer/preprocessed_data/x_train.csv', 'r') as file:
    reader = csv.reader(file)
    x_train = [list(map(int, row)) for row in reader]

with open('/content/drive/MyDrive/transformer/preprocessed_data/x_test.csv', 'r') as file:
    reader = csv.reader(file)
    x_test = [list(map(int, row)) for row in reader]

y_train = []
with open('/content/drive/MyDrive/transformer/preprocessed_data/y_train.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        number = int(row[0])
        y_train.append(number)

y_test = []
with open('/content/drive/MyDrive/transformer/preprocessed_data/y_test.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        number = int(row[0])
        y_test.append(number)

with open('/content/drive/MyDrive/transformer/preprocessed_data/attention_mask_train.csv', 'r') as file:
    reader = csv.reader(file)
    attention_mask_train = [list(map(int, row)) for row in reader]

with open('/content/drive/MyDrive/transformer/preprocessed_data/attention_mask_test.csv', 'r') as file:
    reader = csv.reader(file)
    attention_mask_test = [list(map(int, row)) for row in reader]


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Mounted at /content/drive
Device: cuda


In [2]:
# Loading the training and test data into the device:
x_train = torch.tensor(x_train, dtype=torch.long).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
attention_mask_train = torch.tensor(attention_mask_train, dtype=torch.long).to(device)

x_test = torch.tensor(x_test, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)
attention_mask_test = torch.tensor(attention_mask_test, dtype=torch.long).to(device)

In [3]:
train_data = []
for i in range(len(x_train)):
  train_data.append([x_train[i], attention_mask_train[i], y_train[i]])

test_data = []
for i in range(len(x_test)):
  test_data.append([x_test[i], attention_mask_test[i], y_test[i]])

batch_size = 256

# Using DataLoader to divide training and test data into batches:
data_loader_train = DataLoader(train_data, batch_size = batch_size, shuffle = True)
data_loader_test = DataLoader(test_data, batch_size = batch_size, shuffle = True)

In [4]:
from tqdm import tqdm

# Measuring training and test losses and accuracies of every checkpoint:
train_losses  = []
test_losses  = []
train_accuracy = []
test_accuracy = []

for i in range(11):
  for j in range(2):

    if i == 10 and j == 1:
        break

    model = torch.load(f'/content/drive/MyDrive/transformer/checkpoints/DecoderPosEncWave/DecoderPosEncWave_{i}_{j}.pt')
    model.eval()

    train_losses_temp  = []
    test_losses_temp  = []
    sum_train = 0
    sum_test = 0


    for batch in tqdm(iter(data_loader_train), desc = f'Training data: Epoch {i}, Middle of Epoch: {bool(j)} '):
      x, attention_mask, y = batch
      logits = model(x, attention_mask)

      results = torch.argmax(logits, dim=-1)
      sum_train += torch.sum(results == y).item()

      loss = F.cross_entropy(logits, y)
      train_losses_temp.append(loss.item()*len(batch[0]))

    train_losses.append(sum(train_losses_temp)/len(train_data))
    train_accuracy.append(sum_train/len(train_data))

    for batch in tqdm(iter(data_loader_test), desc = f'Test data:     Epoch {i}, Middle of Epoch: {bool(j)} '):
      x, attention_mask, y = batch
      logits = model(x, attention_mask)

      results = torch.argmax(logits, dim=-1)
      sum_test += torch.sum(results == y).item()

      loss = F.cross_entropy(logits, y)
      test_losses_temp.append(loss.item()*len(batch[0]))

    test_losses.append(sum(test_losses_temp)/len(test_data))
    test_accuracy.append(sum_test/len(test_data))

Test data:     Epoch 10, Middle of Epoch: False : 100%|██████████| 4/4 [00:00<00:00,  4.65it/s]


In [5]:
# Saving losses and accuracies:

with open('/content/drive/MyDrive/transformer/Evaluation/DecoderPosEncWaveTrainLosses.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for number in train_losses:
        writer.writerow([number])

with open('/content/drive/MyDrive/transformer/Evaluation/DecoderPosEncWaveTestLosses.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for number in test_losses:
        writer.writerow([number])

with open('/content/drive/MyDrive/transformer/Evaluation/DecoderPosEncWaveTrainAccuracy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for number in train_accuracy:
        writer.writerow([number])
with open('/content/drive/MyDrive/transformer/Evaluation/DecoderPosEncWaveTestAccuracy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for number in test_accuracy:
        writer.writerow([number])